The model version with hidden infectious <br>
hidden infectious can be switched off! <br>
takes both mu as input parameters --> only for old fits if used for fitting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import minimize

## Useful functions

In [2]:
# various parameters

z = 3
L = 14

#countries = ['Italy', 'Germany', 'Spain', 'United Kingdom', 'Belgium', 'Portugal']
countries = ['Italy', 'Germany', 'Spain', 'United Kingdom']
N_countries = len(countries)
c_index=dict(zip(countries,range(N_countries)))

starts = [15, 20, 18, 22, 24, 25]

new_countries = ['India', 'Brazil', 'Russia', 'US']
new_starts = [25, 18, 23, 23]

# time dependencies
times = ['lock','exp','fermi','rational']

In [3]:
# get_VA
# input: time series of F = I + R = (fraction of) infected + recovered
# output: time series of V and A for given L and z
def get_VA(F, z=z, L=L):
    epsilon=1e-12

    nz = ( F[L:-z] - F[:-L-z] ) / L
    n = ( F[L+z:] - F[z:-L] ) / L

    # 'velocity' (epsilon will avoid log(0))
    v = np.log10 ( n+epsilon )

    # 'acceleration' (epsilon avoids 0/0)
    a = (n-nz) / (z * (n+nz+epsilon)/2)
    return v,a

In [4]:
# get initial parameters for the minimization
def getInitParams(model, a, r, SIRstring=None):
    # load .csv file
    fname = 'init_params.csv'
    if SIRstring: fname = 'init_params_'+SIRstring+'.csv'
    df = pd.read_csv(fname, sep=';')
    
    # find model
    start_ind = (df.Model == model) & (df.a == str(a)) & (df.r == str(r))
    
    try:
        start_ind = np.where(start_ind == True)[0][0]
    except:
        print('You write something wrong!')
        return
    
    # get init params values
    data = np.array( df.iloc[start_ind+2:start_ind+2+N_countries, 1:], dtype=float )
    
    # get params name
    columns = np.array(df.iloc[start_ind+1, : ][1:])
    
    data = pd.DataFrame(data, columns=columns, index=countries)
    
    return data[data.columns.dropna()]

In [5]:
# get optimized parameters
def getOptParams(model, a, r):
    # load .csv file
    fname = 'best_params.csv'
    df = pd.read_csv(fname, sep=';')
    
    # find model
    start_ind = (df.Model == model) & (df.a == str(a)) & (df.r == str(r))
    
    try:
        start_ind = np.where(start_ind == True)[0][0]
    except:
        print('You write something wrong!')
        return
    
    # get init params values
    data = np.array( df.iloc[start_ind+2:start_ind+2+N_countries, 1:], dtype=float )
    
    # get params name
    columns = np.array(df.iloc[start_ind+1, : ][1:])
    
    data = pd.DataFrame(data, columns=columns, index=countries)
    
    data[data.columns.dropna()]
    data=data.drop(columns=['cost','AIC'])
    
    return data

In [6]:
# various plot
def plot_VA(v_data, a_data, v_sim, a_sim, real_lock=None, sim_lock=None, text=None):
    plt.figure(figsize=[10,10])
    plt.plot(v_data, a_data, label='Data')
    plt.plot(v_sim, a_sim, label='Sim. model')
    if not real_lock == None:
        real_lock = int(round(real_lock)) -L -z
        if real_lock -1 >0:
            plt.scatter(v_data[real_lock-1], a_data[real_lock-1], label='Lockdown', c='red')
    if not sim_lock == None:
        sim_lock = int(round(sim_lock)) -L -z
        if sim_lock -1 >0:
            plt.scatter(v_sim[sim_lock-1], a_sim[sim_lock-1], label='Sim. lockdown', c='green')
            
    plt.axhline(y=0, color='gray', linestyle='-.')
    plt.xlabel('v', size=15)
    plt.ylabel('a', size=15)
    plt.legend()
    plt.show()
    
def plot_F(F_data, F_sim, real_lock=None, sim_lock=None):
    plt.figure(figsize=(14, 6))
    plt.plot(range(len(F_data)), F_data, label='Real infected')
    plt.plot(range(len(F_sim)), F_sim, label='Sim. infected')
    if not real_lock == None:
        real_lock = int(round(real_lock))
        plt.axvline(x=real_lock-1, color='red', linestyle='-.', label='Lockdown')
    if not sim_lock == None:
        sim_lock = int(round(sim_lock))
        plt.axvline(x=sim_lock-1, color='green', linestyle='-.', label='Sim. lockdown')
    plt.xlabel('day T')
    plt.ylabel('cumulative number of infected')
    plt.legend()
    plt.show()

In [7]:
# plot predictions
def plotPred(models):
    plt.figure(figsize=(16,18))

    for i, mod in enumerate(models):
        plt.subplot(3,2,i+1)
        v, a = mod.predict(mod.optimal)

        plt.plot(v, a, label='Fit model')
        if mod.time == 'lock':
            opt_t0 = int(mod.optimal[-2])
            plt.scatter(v[opt_t0-L-z-1], a[opt_t0-L-z-1], c='red',marker='X',s=100)
            plt.scatter(v[-1], a[-1], c='magenta',marker='*',s=100,zorder=2)


        plt.plot(mod.v_data, mod.a_data, label='Exp. data')

        plt.scatter(mod.v_data[mod.lock_index-L-z-1], mod.a_data[mod.lock_index-L-z-1], 
                        c='red',marker='X',s=100, label='lockdown date')
        plt.scatter(mod.v_data[-1], mod.a_data[-1], 
                        c='magenta',marker='*',s=100,zorder=2, label='ending date')

        plt.xlim([-7., -3.5])
        plt.axhline(y=0, color='gray', linestyle='-.')
        plt.legend()
        plt.xlabel('v', size=15)
        plt.ylabel('a', size=15)
        plt.title(countries[i], size=18)

    plt.show()

In [8]:
def plotTimeDep(model, a_w, r_w):
    
    # define models
    models = [[None]*len(times)] * len(countries)

    for i, country in enumerate(countries):
        for j, time in enumerate(times):
            models[i][j] = Model(country, model, time=time, start=starts[i])
            print("")
            
    # load fitted weight configurations
    fname = 'best_params.csv'
    df = pd.read_csv(fname, sep=';')
    df.index = df.Model
    a_ws = np.array(df.loc[times[0]].a)
    r_ws = np.array(df.loc[times[0]].r)
    weights=[(a_ws[i],r_ws[i]) for i in range(len(a_ws))]
    
    fig = plt.figure(figsize=(16,30))

    for i in range(len(countries)):
        
        ## V-A plot
        plt.subplot(6,2,2*i+1)

        for j, time in enumerate(times):
            opt_params = getOptParams(model=models[i][j].time,a=a_w,r=r_w).loc[countries[i]]
            v, a = models[i][j].predict(opt_params)

            plt.plot(v, a, label=time+' time')
            plt.scatter(v[-1], a[-1], 
                            c='magenta',marker='*',s=100,zorder=2)

        # Experimental data
        plt.plot(models[i][0].v_data, models[i][0].a_data, '-.', label='Data')

        plt.scatter(models[i][0].v_data[models[i][0].lock_index-L-z-1], 
                    models[i][0].a_data[models[i][0].lock_index-L-z-1], 
                    c='red',marker='X',s=100, label='lockdown date')
        plt.scatter(models[i][0].v_data[-1], models[i][0].a_data[-1], 
                        c='magenta',marker='*',s=100,zorder=2, label='ending date')

        plt.xlim([-7., -3.5])
        plt.axhline(y=0, color='gray', linestyle='-.')
        plt.legend()
        plt.xlabel('v', size=14)
        plt.ylabel('a', size=14)
        plt.title(countries[i]+' - V vs. A plot', size=18)

        ## Beta
        plt.subplot(6,2,2*i+2)

        t = np.arange(0, m_exp[i].T)

        # Exponential
        for j, time in enumerate(times):
            opt_params = getOptParams(model=models[i][j].time,a=a_w,r=r_w).loc[countries[i]]
            beta = models[i][j].predict(opt_params, out='Beta')

            plt.plot(t, beta, label=time+' time')
        
        plt.axvline(x=models[i][0].lock_index, color='grey', linestyle='-.', label='lockdown date')

        plt.legend()
        plt.xlabel('t', size=14)
        plt.ylabel(r'$\beta(t)$', size=14)
        plt.title(countries[i]+' - beta vs. t plot', size=18)

    fig.subplots_adjust(hspace=.3)
    plt.show()
    

In [9]:
def plotWeights(model, time): #all models have same time dependency
    
    # define models
    models = []
    for i, country in enumerate(countries):
        models.append( Model(country, model, time=time, start=starts[i]) )
        print("")
    
    # load fitted weight configurations
    fname = 'best_params.csv'
    df = pd.read_csv(fname, sep=';')
    df.index = df.Model
    a_ws = np.array(df.loc[time].a)
    r_ws = np.array(df.loc[time].r)
    weights=[(a_ws[i],r_ws[i]) for i in range(len(a_ws))]

    # PLOT
    fig = plt.figure(figsize=(16,30))
    
    for i in range(len(countries)):
        ## V-A plot
        plt.subplot(6,2,2*i+1)

        for a_w,r_w in weights:
            opt_params = getOptParams(model=time,a=a_w,r=r_w).loc[countries[i]]
            v, a = models[i].predict(opt_params)

            plt.plot(v, a, label='a='+a_w+' / r='+r_w)
            plt.scatter(v[-1], a[-1], 
                            c='magenta',marker='*',s=100,zorder=2)

        #experimental Data
        plt.plot(models[i].v_data, models[i].a_data, '-.', label='Data')
        plt.scatter(models[i].v_data[models[i].lock_index-L-z-1], models[i].a_data[models[i].lock_index-L-z-1], 
                        c='red',marker='X',s=100, label='lockdown date')
        plt.scatter(models[i].v_data[-1], models[i].a_data[-1], 
                        c='magenta',marker='*',s=100,zorder=2, label='ending date')

        plt.xlim([-7., -3.5])
        plt.axhline(y=0, color='gray', linestyle='-.')
        plt.legend()
        plt.xlabel('v', size=14)
        plt.ylabel('a', size=14)
        plt.title(time+' - '+countries[i]+' - V vs. A plot', size=18)

        ##Beta Plot
        plt.subplot(6,2,2*i+2)

        t = np.arange(0, models[i].T)

        for a_w,r_w in weights:
            opt_params = getOptParams(model=time,a=a_w,r=r_w).loc[countries[i]]
            beta = models[i].predict(opt_params, out='Beta')
            plt.plot(t, beta, label='a='+a_w+' / r='+r_w)

        plt.axvline(x=models[i].lock_index, color='grey', linestyle='-.', label='lockdown date')
        plt.legend()
        plt.xlabel('t', size=14)
        plt.ylabel(r'$\beta(t)$', size=14)
        plt.title(countries[i]+' -' + time + ' beta vs. t plot', size=18)
    fig.subplots_adjust(hspace=.3)
    plt.show()
    return

In [10]:
# print optimized parameters
def printOptimized(models, params):
    params.append('cost')
    params.append('AIC')
    
    data = np.zeros(( N_countries, len(params) ))

    for i,mod in enumerate(models):
        data[i,:-2] = np.array(mod.optimal)
        data[i,-2] = mod.cost_function(np.sqrt(mod.optimal))
        data[i,-1] = mod.AIC(np.sqrt(mod.optimal))

    best_params = pd.DataFrame(data, columns=params, index=countries)
    if 't_lock' in params:
        best_params['t_lock'] = best_params['t_lock'].astype('int')

    return best_params

## Model class

- Cost function:<br>
$$ C(\theta) = \dfrac{\sum_t [w_t(v_t - \hat{v_t})^2 + w'_t(a_t - \hat{a_t})^2]}{\sum_t w_t + w'_t} $$
** the cost function is equal to the weighted RMSE -> $C(\theta)=\sigma^2$
<br><br>
- Weights:
$$ w = n^r $$ $$ w' = a * w = a * n^r $$
<br><br>
- Log-likelihood:<br>
$$ \ln{L} = - \dfrac{n}{2} ( \ln{2\pi\sigma^2} +1 ) = - T \ln{\sigma^2} + const.  $$
where n = 2T = size of data sample<br>
(https://en.wikipedia.org/wiki/Maximum_likelihood_estimation)
<br><br>
- AIC:
$$ AIC = 2k - 2\ln{L} = 2k + 2T\ln{C(\theta)}$$

In [11]:
# Model class
class Model(object):
    '''
    - init: define country name, model, starting day, state -> automatically load data
            where state refers to state "bl" -> before lockdown; state "al" -> after lockdown
    - fit: define initial parameters and whether fit (v,a) or F -> return best model
    - predict: define how long you want to predict
    '''
    
    def __init__(self, country, model, time=None, start=None, end='6/30/20', state=None, hidden=True, download=True):
        '''
        Class initializer.
        
        Params:
        - country: name of the country to be analyzed
        - model: epidemiological model to fit data (SIR, SIAR, SEIAR)
        - time: time-dependecy of infectious rate
                default -> constant infectious rate
        - start: number of days to be discarded
                 default -> start from day 0
        - end: final date of the analysis
                 default -> May 31st 2020
        - state: which part of the time series is going to be analyzed
            * None: whole time series
            * 'bl': only data before lockdown
            * 'al': only data after lockdown
        '''
        
        print('Creating '+str(model.__name__)+' for '+str(country))
        
        self.country = country
        self.model = model
        self.time = time
        self.start = start
        self.end = end
        self.hidden = hidden #hide asymptomatics from data in fit and prediction --> hidden = true
        
        self.download = download #if no internet
        
        self.load_population()
        self.load_data()
        self.load_lockdown()        
        
        self.state = state
        if not state == None:
            self.restrict_data()
        
        
    def load_population(self):
        '''
        Loading of the country's population.
        '''
        
        # Url of the dataset with country's population from 1960 to 2018
        url = 'https://raw.githubusercontent.com/datasets/population/master/data/population.csv'
        
        # Manage misleading contries' name in the database
        c_name = self.country
        if self.country == 'US':
            c_name = 'United States'
        elif self.country == 'Iran':
            c_name = 'Iran, Islamic Rep.'
        elif self.country == 'Russia':
            c_name = 'Russian Federation'
        
        try:
            # Read dataset as .csv
            if self.download: df = pd.read_csv(url,index_col=1)
            else: df = pd.read_csv('DATA/population.csv', index_col=0)
            # Extract population of last year
            self.N = df[ df['Country Name'] == c_name ].iloc[-1].Value 
            print('Loaded country\'s population')
            
        except:
            # Set default value as 10 million
            print('Unable to load country\'s population, N = 10e6 is set by default')
            self.N = 10000000
    
    
    def load_data(self):
        '''
        Loading of cumulative infected data for all country.
        
        Columns: Country, Province, Lat, Long, 22/1/20, 23/1/20, ...
        '''
        
        # Url of the dataset with cumulative infected data starting from 22/1/2020
        url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
        
        try:
            # Read dataset as .csv
            if self.download: df = pd.read_csv(url,index_col=1)
            #load data from DATA folder
            else: df = pd.read_csv('DATA/infected.csv', index_col=0)
            # Extract time series of selected country
            self.country_df = df[ df.index == self.country ]
            # Keep data only until end date
            if not self.end == None:
                end_index = df.columns.get_loc(self.end)
                self.country_df = self.country_df.iloc[:,:end_index+1]
            # Sum regional values (if present).
            # First three columns of our dataset are Province, lat and Long, so we consider the data from 3rd column
            self.data = np.array(self.country_df.iloc[:,3:], dtype=float).sum(axis=0)
            
            # Consider data from user-defined starting day (if defined)
            if not self.start == None:
                # Note: self.start is an index and not date-time format
                self.data = self.data[self.start:]
            
            # Compute time series of n and v,a
            self.T = len(self.data)
            self.F = self.data / self.N
            self.v_data, self.a_data = get_VA(self.F)
            self.T_va = len(self.v_data)
            self.n = ( self.F[L+z:] - self.F[z:-L] ) / L
            print('Loaded time series of '+str(self.T)+' days')
            
        except:
            print('Unable to load confirmed data')
            
    
    def load_lockdown(self):
        '''
        Loading of lockdown date.
        '''
        
        # Url of the dataset with lockdown date for each country
        #url = 'https://raw.githubusercontent.com/nevoht/LoCP/master/countryLockdowndatesJHUMatch.csv'
        url = 'https://raw.githubusercontent.com/GiacomoBarzon/Covid-19/master/lockdowns.csv'
        
        try:
            # Read dataset as .csv
            if self.download: df_lockdown = pd.read_csv(url,usecols=('Country/Region','Date'))
            else: df_lockdown = pd.read_csv('DATA/lockdowns.csv',index_col=0)
            # Extract lockdown date of selected country
            df_lockdown.index = df_lockdown['Country/Region']
            self.lockdown = df_lockdown.loc[self.country][1]
            date=str(self.lockdown)
            
            if date == 'nan':
                print('Lockdown date not found')
                self.lock_index = None
                return
            
            # Convert date format
            month=date[6:7]
            day=date[-2:]
            date=month+'/'+ day + '/20'
            
            # Get index of lockdown date
            # here we subtract 3 as we counted 1st 3 columns which are not dates i.e. province, lat, long 
            self.lock_index = self.country_df.columns.get_loc(date) - 3
                
            print('Lockdown at '+str(self.lockdown)+' -> at day '+str(self.lock_index))
            
            # Subtract starting day
            if not self.start == None:
                # if we started from some other date other than 22.1.20, we need to subtract it from the index
                self.lock_index -= self.start
                if self.lock_index <0:
                    self.lock_index = None
            print('first VA index with lockdown:', self.lock_index-L-z)
            
        except:
            print('Unable to load lockdown data')


    def restrict_data(self):
        '''
        Restrict time series to value only data before or after lockdown (if defined).
        '''
        
        if not self.lock_index == None:
            if self.state == 'bl':
                self.data = self.data[:self.lock_index]
            elif self.state == 'al':
                self.data = self.data[self.lock_index-L-z:]
                
        # Compute time series of n and v,a
        self.T = len(self.data)
        self.F = self.data / self.N
        self.v_data, self.a_data = get_VA(self.F)
        self.T_va = len(self.v_data)
        self.n = ( self.F[L+z:] - self.F[z:-L] ) / L
                    
            
    def cost_function(self, params):
        '''
        Cost function used for the optimization.
        
        Two different cost function are defined, wheter we want to fif orignal time series F or v-a data
        based on self.fitOriginalData.
        
        For the cost function of v-a data, we use also some weights defined in the self.fit function:
        - w = n^r -> weights over the velocities 
        - w_prime = a*w = a*n^r -> weights over the accelerations
        '''
        
        if self.state == 'al':
            lock = self.lock_index
        else:
            lock = None
        
        # Cost function for F
        if self.fitOriginalData == True:
            Fsim = self.model(params, self.time, self.T, lock, out='F', hidden=self.hidden)
            
            return np.mean( (Fsim-self.F)**2 )
        
        # Cost function for v-a
        else:
            v,a = self.model(params, self.time, self.T, lock, hidden=self.hidden)
            
            # weights
            w = self.n**self.r
            w_prime = self.a * w
           
            return np.sum( w*(v-self.v_data)**2 + w_prime*(a-self.a_data)**2 ) / np.sum(w+w_prime)
        
        
    def AIC(self, params):
        '''
        Akaike information criterion.
        
        AIC = 2k - 2 ln(L)
            = 2k + N ln(C)
        
        where k = number of parameters in the model
              L = likelihood
              C = cost function
              
        ** for small sample size it's better to use the corrected AIC:
        AICc = AIC + (2k^2 + 2k)/(n -k -1)
        '''
        
        if self.state == 'al':
            lock = self.lock_index
        else:
            lock = None
        
        # total number of parameters in the model
        k = len(params)
        
        # cost function -> respect to raw data
        Fsim = self.model(params, self.time, self.T, lock, out='F', hidden=self.hidden)
        cost = np.mean( (Fsim-self.F)**2 )
        
        return 2 * k + self.T * np.log(cost)
    
    
    def fit(self, init_params, method=None, bounds=None, fitOriginalData=False, debug=False, r=0., a=1., printer=True):
        '''
        Fit the model to the data in order to find the best parameters.
        
        Function 'scipy.optimize.minimize' is used for the optimization.
        
        Params:
        - method: optimization algorithm as defined in 'scipy.optimize.minimize'
                  default -> 'Nelder-Mead' method is used if not specified
        - bounds: bounds on the parameters during the minimization
                  BUT not defined for all the minimization algorithms
        - fitOiginalData: if false ->  fit v-a time series
                          if true -> fit F time series
        - r, a: parameters of the weight
        '''
        
        # Extract params
        self.fitOriginalData = fitOriginalData
        self.r = r
        self.a = a
        
        # Fit data
        if printer:
            print('Start fitting')
        
        # taking the square-root in order to have positive parameters
        init_params = np.sqrt(init_params)
        
        if debug == True:
            optimal = minimize(self.cost_function, init_params, method='Nelder-Mead', 
                               options={'adaptive': True})
        else:
            if method == None:
                optimal = minimize(self.cost_function, init_params, method='Nelder-Mead', 
                                   options={'adaptive': True, 'maxiter': len(init_params)*1000, 'maxfev': len(init_params)*1000})
            else:
                optimal = minimize(self.cost_function, init_params, method=method, bounds=bounds)

        if printer:
            print('End fitting')
            print('Results:')
            print(optimal)
        self.optimal = np.array(optimal.x)**2
        self.success = optimal.success
    
    
    def predict(self, params, T=None, out=None):
        '''
        Predict the output of the model.
        
        Params:
        - params: parameters of the model
                  e.g. params = self.optimal
        - T: total number of days to be predicted
             default -> same number of the real time series
        - outSIR: if false -> return predicted v-a 
                  if true -> return predicted (S,I,R)
        '''
        
        # taking the square-root
        params = np.sqrt(params)
        
        if T==None:
            T = self.T
            
        if self.state == 'al':
            lock = self.lock_index
        else:
            lock = None
        
        return self.model(params, self.time, T, lock, out, hidden = self.hidden)
    

# Time dependencies
- Lockdown:
\begin{align}
\beta(t) & = \beta & for \ t<t_{lock} \\
\beta(t) & = \beta_{lock} & for \ t>t_{lock} \\
\end{align}
- Decreasing exponential:
$$ \beta(t) = \beta(0) * \exp(- \alpha t) $$
- Fermi function:
$$ \beta(t) = \beta(0) * \dfrac{ 2 }{ 1 + \exp(\alpha (t-t_0)) }   $$
- Rational:
$$ \beta(t) = \beta(0) * \dfrac{1}{1+(\alpha t)^2}   $$

## SIR

In [12]:
# SIR model
#
# params: square root of 
#          [beta, mu,
#          initial fraction of infected I0, 
#          total number of days of simulation T]
#
# outSIR: if true -> return S[t], I[t], R[t]
#         if false -> return v[t], a[t]

def SIR(params, time, T, lock, out=None, hidden=True):#hidden has no meaning for SIR
        
    # unpack parameters
    t = np.arange(0,T)
    
    if not lock == None:
        t += lock
    
    if time == 'lock':
        beta, mu, I0, t_lock, beta_lock = np.array(params)**2
        t_lock = int(t_lock)
        beta_t = np.zeros(T) + beta
        beta_t[t_lock:] = beta_lock
    elif time == 'exp':
        beta, mu, I0, alpha = np.array(params)**2
        beta_t = beta * np.exp(-alpha*t)
    elif time == 'fermi':
        beta, mu, I0, alpha, t0 = np.array(params)**2
        beta_t = beta * 2. / (1. + np.exp(alpha*(t-t0)))
    elif time == 'rational':
        beta, mu , I0, alpha = np.array(params)**2
        beta_t = np.divide(beta, 1.+(alpha*t)**2)
    else: #constant
        beta, mu, I0 = np.array(params)**2
        beta_t = np.zeros(T) + beta
    
    # integrating time
    dt = 1./24.
    
    # define initial conditions
    S0 = 1. - I0
    R0 = 0.
    
    # create state array
    y = np.zeros((T, 3))
    y[0] = [S0, I0, R0]
    
    # loop over days
    for t in range(1,T):
        dy = np.array(y[t-1])
        
        # loop over hours
        for h in range(24):
            S, I, R = dy
            if S<0 or I<0 or R<0:
                print('ERROR: Something becomes negative')
                break
            dy += np.array(([-beta_t[t]*S*I, beta_t[t]*S*I-mu*I, mu*I])) * dt
            
        y[t] = dy
                        
    if out == None:
        return get_VA( y[:,1]+y[:,2] )
    elif out == 'Beta':
        return beta_t
    elif out == 'F':
        return y[:,1] + y[:,2]
        

## SIAR

In [13]:
def SIAR(params, time, T, lock, out=None, hidden=True):
    
    # unpack parameters
    t = np.arange(0,T)
    
    if not lock == None:
        t += lock
    
    if time == 'lock':
        beta_I, beta_A, mu_I, mu_A, nu, I0, t_lock, beta_lock = np.array(params)**2
        t_lock = int(t_lock)
        beta_I = np.zeros(T) + beta_I
        beta_I[t_lock:] = beta_lock
        beta_A = np.zeros(T) + beta_A
        beta_A[t_lock:] = beta_lock
  
    elif time == 'exp':
        beta_I, beta_A, mu_I, mu_A, nu, I0, alpha = np.array(params)**2
        beta_I = beta_I * np.exp(-alpha*t)
        beta_A = beta_A * np.exp(-alpha*t)
    elif time == 'fermi':
        beta_I, beta_A, mu_I, mu_A, nu, I0, alpha, t0 = np.array(params)**2
        beta_I = beta_I * 2. / (1. + np.exp(alpha*(t-t0)) )
        beta_A = beta_A * 2. / (1. + np.exp(alpha*(t-t0)))
    elif time == 'rational':
        beta_I, beta_A, mu_I, mu_A, nu, I0, alpha = np.array(params)**2
        beta_I = np.divide(beta_I, 1.+(alpha*t)**2)
        beta_A = np.divide(beta_A, 1.+(alpha*t)**2)
    else: #constant
        beta_I, beta_A, mu_I, mu_A, nu, I0 = np.array(params)**2
        beta_I = np.zeros(T) + beta_I
        beta_A = np.zeros(T) + beta_A
   
    
    dt = 1./24.
    
    # define initial conditions
    S0 = 1. - I0
    R0 = 0.
    A0 = 0.
    
    y = np.zeros((T, 4))
    y[0] = [S0, I0, A0, R0]
    
    # loop over days
    for t in range(1,T):
        dy = np.array(y[t-1])
        
        # loop over hours
        for h in range(24):
            S, I, A, R = dy
            if S<0 or I<0 or R<0 or A<0:
                print('ERROR: Something becomes negative')
                break
            
            ds = -(beta_I[t]+beta_A[t]) * (I+A) * S
            di = beta_I[t]*(I+A)*S - mu_I*I + nu*A
            da = beta_A[t]*(I+A)*S - (mu_A+nu)*A
            dr = mu_I*I + mu_A*A
            
            dy += np.array( [ds, di, da, dr] ) * dt
            
        y[t] = dy
        
    if out == None:
        if hidden:
            return get_VA( y[:,1] + y[:,3] ) #Baiesi's point: fit without asymptomatics
        else:
            return get_VA( y[:,1] + y[:,2] + y[:,3] )
    elif out == 'Beta':
        return beta_I
    elif out == 'F':
        if hidden:
            return y[:,1] + y[:,3] #Baiesi's point: fit without asymptomatics
        else:
            return y[:,1] + y[:,2] + y[:,3] 

## SEIAR

In [14]:
def SEIAR(params, time, T, lock=None, out=None, hidden=True):
    t = np.arange(0,T)  #define timesteps

    if lock != None:
        t += lock
    
    if time == 'lock':
        beta, gamma, p, mu_I, mu_A, I0, t_lock, beta_lock = np.array(params)**2
        t_lock = int(t_lock)
        beta_t = np.zeros(T) + beta
        beta_t[t_lock:] = beta_lock
    elif time == 'exp':
        beta, gamma, p, mu_I, mu_A, I0, alpha  = np.array(params)**2
        #beta, gamma, p, mu_I, mu_A, I0, alpha = np.array(params)**2
        beta_t = beta * np.exp(-alpha*t)
    elif time == 'fermi':
        #beta, gamma, p, mu_I, mu_A, I0, alpha, t0, beta_0 = np.array(params)**2
        beta, gamma, p, mu_I, mu_A, I0, alpha, t0 = np.array(params)**2
        #beta_t = beta*(1- np.divide(1,1+np.exp(-alpha*(t-t0))))
        beta_t = beta * 2. / ( 1. + np.exp(alpha*(t-t0)) )
    elif time == 'rational':
        beta, gamma, p, mu_I, mu_A, I0, alpha = np.array(params)**2
        eps_beta = 1.9
        #beta, gamma, p, mu_I, mu_A, I0, alpha, eps_beta = np.array(params)**2
        beta_t = np.divide(beta, 1. + (alpha * t)**eps_beta)
    else: #constant
        beta, gamma, p, mu_I, mu_A, I0 = np.array(params)**2
        beta_t = np.zeros(T) + beta
        
    #p=0.43 #if we trust the Vo result
    
    dt = 1./24.
    
    # define initial conditions
    
    E0=I0/p #probably more asymptomatic people in the beginning
    A0=E0-I0
    S0=1-(E0+I0+A0)
    E0,A0,R0 = 0.,0.,0.
    
    y = np.zeros((T, 5))
    y[0] = [S0, E0, I0, A0, R0]
    
    # loop over days
    for t in range(1,T):
        dy = np.array(y[t-1])
        
        # loop over hours
        for h in range(24):
            S, E, I, A, R = dy
            if S<0 or I<0 or R<0 or A<0 or E<0:
                break
                        
            ds = -beta_t[t]*(I+A)*S
            de = beta_t[t]*(I+A)*S - gamma*E
            di = gamma*p*E - mu_I*I
            da = gamma*(1.-p)*E - mu_A*A
            dr = mu_I*I +mu_A*A
            
            dy += np.array(( [ds, de, di, da, dr] )) * dt #dt--> rates per day
            
        y[t] = dy

        
    if out == None:
        if hidden:
            return get_VA( y[:,2] + y[:,4] ) #Baiesi's point: fit without asymptomatics
        else:
            return get_VA( y[:,2] + y[:,3] + y[:,4] )
    elif out == 'Beta':
        return beta_t
    elif out == 'F':
        if hidden:
            return y[:,2] + y[:,4] #Baiesi's point: fit without asymptomatics
        else:
            return y[:,2] + y[:,3] + y[:,4]

# SHIR

In [15]:
def SHIR(params, time, T, lock, out=None, hidden=True):
    
    # unpack parameters
    t = np.arange(0,T)
    
    if not lock == None:
        t += lock
    
    beta, mu, I0, sigma = np.array(params)**2
    beta = np.zeros(T) + beta
    
    dt = 1./24.
    
    # define initial conditions
    S0 = 1. - I0
    R0 = 0.
    H0 = 0.
    
    y = np.zeros((T, 4))
    y[0] = [S0, H0, I0, R0]
    
    # loop over days
    for t in range(1,T):
        dy = np.array(y[t-1])
        
        # loop over hours
        for h in range(24):
            S, H, I, R = dy
            if S<0 or H<0 or I<0 or R<0:
                print('ERROR: Something becomes negative')
                break
                
            ds = -beta[t]*I*S -sigma*S
            dh = sigma*S
            di = beta[t]*I*S - mu*I
            dr = mu*I
            
            dy += np.array( [ds, dh, di, dr] ) * dt
            
        y[t] = dy
        
    if out == None:
        return get_VA( y[:,2] + y[:,3] )
    elif out == 'Beta':
        return beta_I
    elif out == 'F':
        return y[:,2] + y[:,3]

In [16]:
def SSHIR(params, time, T, lock, out=None, hidden=True):
    
    # unpack parameters
    t = np.arange(0,T)
    
    if not lock == None:
        t += lock
    
    beta, mu, I0, sigma, p = np.array(params)**2
    beta = np.zeros(T) + beta
    
    dt = 1./24.
    
    # define initial conditions
    S0 = 1. - I0
    Sprime0 = S0 * p
    S0 = S0 * (1.-p)
    R0 = 0.
    H0 = 0.
    
    y = np.zeros((T, 5))
    y[0] = [S0, Sprime0, H0, I0, R0]
    
    # loop over days
    for t in range(1,T):
        dy = np.array(y[t-1])
        
        # loop over hours
        for h in range(24):
            S, Sprime, H, I, R = dy
            if S<0 or Sprime<0 or H<0 or I<0 or R<0:
                print('ERROR: Something becomes negative')
                break
                
            ds = -beta[t]*I*S
            dsprime = -beta[t]*I*Sprime -sigma*Sprime
            dh = sigma*Sprime
            di = beta[t]*I*(S+Sprime) - mu*I
            dr = mu*I
            
            dy += np.array( [ds, dsprime, dh, di, dr] ) * dt
            
        y[t] = dy
        
    if out == None:
        return get_VA( y[:,3] + y[:,4] )
    elif out == 'Beta':
        return beta_I
    elif out == 'F':
        return y[:,3] + y[:,4]